## 各種import

In [1]:
from io import StringIO
from pyxcp.config import readConfiguration
from pyxcp import transport
from pyxcp.transport.candriver.pc_vector import Vector # "CAN_DRIVER": "Vector"を指定する場合に必要
from pyxcp.master import Master
from collections import namedtuple
import time

## コンフィグレーションパラメータ(JSON文字列)の作成と読み込み

In [2]:
JSON = """{
"TRANSPORT": "CAN",
"CAN_DRIVER": "Vector",
"CAN_USE_DEFAULT_LISTENER": true,
"CHANNEL": "0",
"CAN_ID_MASTER": 2,
"CAN_ID_SLAVE": 1,
"CAN_ID_BROADCAST": 256,
"MAX_DLC_REQUIRED": false,
"CREATE_DAQ_TIMESTAMPS": true,
"SERIAL": 0,
"BITRATE": 500000,
"APP_NAME": "PyXCP-CAN"
}"""
CONF_JSON = StringIO(JSON)
CONF_JSON.name = "conf_can_vector.json"
CONFIG = readConfiguration(CONF_JSON)

## コンフィグレーションを元にXCPマスターの生成

In [3]:
xm=Master('can',config=CONFIG)

## CONNECT

In [4]:
conn = xm.connect()
print(conn)

Container: 
    resource = Container: 
        dbg = False
        pgm = False
        stim = False
        daq = True
        calpag = False
    commModeBasic = Container: 
        optional = True
        slaveBlockMode = False
        addressGranularity = (enum) BYTE 0
        byteOrder = (enum) INTEL 0
    maxCto = 8
    maxDto = 8
    protocolLayerVersion = 1
    transportLayerVersion = 1


In [5]:
print("calpag =", xm.slaveProperties.supportsCalpag)
print("daq =", xm.slaveProperties.supportsDaq)
print("pgm =", xm.slaveProperties.supportsPgm)
print("stim =", xm.slaveProperties.supportsStim)

calpag = False
daq = True
pgm = False
stim = False


## GET_STATUS

In [6]:
status=xm.getStatus()
print(status)

Container: 
    sessionStatus = Container: 
        resume = False
        daqRunning = False
        clearDaqRequest = False
        storeDaqRequest = False
        storeCalRequest = False
    resourceProtectionStatus = Container: 
        dbg = False
        pgm = False
        stim = False
        daq = False
        calpag = False
    sessionConfiguration = 0


## SYNCH

In [7]:
res=xm.synch()
print(res)

bytearray(b'\x00')


## GET_COMM_MODE_INFO

In [8]:
CommModeInfo=xm.getCommModeInfo()
print(CommModeInfo)

Container: 
    commModeOptional = Container: 
        interleavedMode = False
        masterBlockMode = False
    maxBs = 0
    minSt = 0
    queueSize = 0
    xcpDriverVersionNumber = 16


## DOWNLOAD、UPLOADによるverify

In [9]:
xm.setMta(0x0000,0xff)
bhv = xm.upload(4)
print(bhv.hex())

00000000


In [10]:
bhv = bytearray(bhv)
bhv[0] = 0xAA
bhv[1] = 0x55
xm.setMta(0x0000,0xff)
xm.download(bhv)

bytearray(b'')

In [11]:
xm.setMta(0x0000,0xff)
bhv = xm.upload(4)
print(bhv.hex())

aa550000


## DAQリストの構築

In [12]:
xm.freeDaq();

In [13]:
xm.allocDaq(1);

In [14]:
xm.allocOdt(0, 1);

In [15]:
xm.allocOdtEntry(0, 0, 1);

## ODT_ENTRYへの書き込み

In [16]:
DaqEntry = namedtuple("DaqEntry", "daq odt entry bitoff size ext addr")
de0 = (
    DaqEntry(daq=0, odt=0,  entry=0, bitoff=255, size=4, ext=0xff, addr=0x00000004),
)


In [17]:
for daq, odt, entry, bitoff, size, ext, addr in de0:
    xm.setDaqPtr(daq, odt,  entry)
    xm.writeDaq(bitoff, size, ext, addr)

## DAQの起動

In [18]:
xm.setDaqListMode(0x10, 0, 1, 1, 0)
res = xm.startStopDaqList(0x02, 0)
print("startStopDaqList #0", res)

startStopDaqList #0 Container: 
    firstPid = 0


In [19]:
xm.startStopSynch(0x01);

## DAQの計測とDOWNLOADによる外部からの値更新

In [20]:
download_wait = 0.05
start = time.time()
download_time =  time.time()+download_wait-0.03

ram_update = [
    [0x11, 0x22, 0x33, 0x44],
    [0x55, 0x66, 0x77, 0x88],
    [0x99, 0xAA, 0xBB, 0xCC],
    [0x00, 0x00, 0x00, 0x00],
]
i = 0
str = ""
while True:
    queue_len = len(xm.transport.daqQueue)
    for _ in range(queue_len):
        daq = xm.transport.daqQueue.popleft()
        str += '%.6f, %s\n' % ( daq[3],daq[0].hex() )
    if time.time() > download_time:
        xm.setMta(0x00000004,0xff)
        bhv = bytearray(ram_update[i])
        xm.download(bhv)
        str += 'download %s\n' % bhv.hex()
        download_time =  download_time + download_wait
        i = i+1
        if i > 3:
            i = 3
    if time.time() > start + 0.22:
        break
    time.sleep(0.010)
    
print(str)

1637617964.960732, 00000000000000
1637617964.970694, 00630000000000
1637617964.980664, 00c70000000000
1637617964.990625, 002a0100000000
1637617965.000587, 008e0100000000
1637617965.010556, 00f20100000000
1637617965.020526, 00550200000000
1637617965.031487, 00c30200000000
download 11223344
1637617965.041465, 00270300000000
1637617965.051442, 008b0300000000
1637617965.061437, 00ee0311223344
1637617965.071455, 00530411223344
1637617965.081409, 00b60411223344
download 55667788
1637617965.091411, 001a0511223344
1637617965.101348, 007e0511223344
1637617965.111318, 00e10555667788
1637617965.121296, 00450655667788
1637617965.131257, 00a90655667788
download 99aabbcc
1637617965.141219, 000c0755667788
1637617965.151229, 00700755667788
1637617965.161248, 00d50799aabbcc
1637617965.171185, 00380899aabbcc
1637617965.181204, 009c0899aabbcc
1637617965.191124, 00ff0899aabbcc
download 00000000
1637617965.201077, 00630999aabbcc
1637617965.211047, 00c70999aabbcc
1637617965.221000, 002a0a00000000
1637617965

In [21]:
xm.startStopSynch(0x00);

In [22]:
xm.disconnect();